In [1]:
# In this we are going to see how we can push out model to the HuggingFace hub with code

In [ ]:
from huggingface_hub import notebook_login
notebook_login() # thsi function helps us to loggig in our huggingFace account

In [3]:
#Now code
from datasets import load_dataset
raw_dataset = load_dataset("glue","cola")

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [5]:
from transformers import AutoTokenizer
model_checkpoint= "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
def preprocess_function(example):
    return tokenizer(example['sentence'],truncation=True)
tokenized_dataset = raw_dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import numpy as np
from datasets import load_metric

In [9]:
metric = load_dataset("glue","cola")

def compute_metrics(eval_pred):
    predictions,labels=eval_pred
    predictions=np.argmax(predictions,axis=-1)
    return metric.compute(predictions=predictions,references=lables)

In [10]:
from transformers import TrainingArguments
args = TrainingArguments(
    "bert-fine-tuned-cola",
    evaluation_strategy="epoch",
    save_strategy="epch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import Trainer
trainer=Trainer(
    model,
    args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)
trainer.train()

In [ ]:
trainer.push_to_hub("end of training")

# Pushing components individauly

In [ ]:
repo_name="bert-fine-tuned-cola"
model.psuh_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

# Labels

In [12]:
label_names = raw_dataset['train'].features['label'].names
label_names

['unacceptable', 'acceptable']

In [ ]:
model.config.id2label = {str(i):lb1 for l ,lb1 in enumerate(label_names)}
model.config.label2id = {lbl:str(i) for l ,lb1 in enumerate(label_names)}

In [ ]:
repo_name="bert-fine-tuned-cola"
model.config.push_to_hub(repo_name)

# Push to Hub (Tensorflow)

In [ ]:
# first logging into your account
from huggingface_hub import notebook_login
notebook_login()

In [1]:
#loading the datasets
from datasets import load_dataset
raw_dataset=load_dataset("glue","cola")
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [2]:
from transformers import AutoTokenizer
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
def preprocess_fucntion(example):
    return tokenizer(example["sentence"],truncation=True)
tokenized_dataset = raw_dataset.map(preprocess_fucntion,batched=True)
tokenized_dataset

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1063
    })
})

In [5]:
from transformers import DataCollatorWithPadding
collator = DataCollatorWithPadding(tokenizer=tokenizer,return_tensors='tf')

train_dataset = tokenized_dataset['train'].to_tf_dataset(
    columns=['attention_mask','input_ids','labels','token_type_ids'],
    collate_fn=collator,
    batch_size=32,
    shuffle=True
)

validation_dataset = tokenized_dataset['validation'].to_tf_dataset(
    columns=['attention_mask','input_ids','labels','token_type_ids'],
    collate_fn=collator,
    batch_size=32,
    shuffle=False
)

In [6]:
#loading the model
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import AdamWeightDecay
optimizer = AdamWeightDecay(2e-5,weight_decay_rate=0.01)
model.compile(optimizer=optimizer)

In [ ]:
from transformers import PushToHubCallback
callbacks = [PushToHubCallback("model_output/",
                               tokenizer=tokenizer,
                               hub_model_id="bert-fine-tuned-cola")
model.fit(train_dataset,validation_dataset=validation_dataset,epochs=2,callbacks=callable)

In [ ]:
# now push the model to the hub
model.push_to_hub("bert-fine-tuned-cola",commit_message="End of Traning")

# Lables

In [9]:
label_names = raw_dataset['train'].features['label'].names
label_names

['unacceptable', 'acceptable']

In [ ]:
model.config.id2label = {str(i):lb1 for l ,lb1 in enumerate(label_names)}
model.config.label2id = {lbl:str(i) for l ,lb1 in enumerate(label_names)}

In [ ]:
repo_name="bert-fine-tuned-cola"
model.config.push_to_hub(repo_name)

In [ ]:
# For Loading the pushed model
loaded_model = TFAutoModelForSequenceClassification.from_pretrained("full repo path")